In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-27 16:47:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.77MB/s    in 0.5s    

2021-09-27 16:47:45 (1.77 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
cameras_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True)
cameras_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

Drop na and duplicates

In [8]:
cameras_df = cameras_df.dropna()
cameras_df = cameras_df.dropDuplicates()

Schema - Cameras_df

In [9]:
cameras_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Recast customer_id, product_parent, star_rating, helpful_votes, total_votes as IntegerType

In [10]:
from pyspark.sql.types import IntegerType
cameras_df = cameras_df.withColumn("customer_id",cameras_df["customer_id"].cast(IntegerType()))
cameras_df = cameras_df.withColumn("product_parent",cameras_df["product_parent"].cast(IntegerType()))
cameras_df = cameras_df.withColumn("star_rating",cameras_df["star_rating"].cast(IntegerType()))
cameras_df = cameras_df.withColumn("helpful_votes",cameras_df["helpful_votes"].cast(IntegerType()))
cameras_df = cameras_df.withColumn("total_votes",cameras_df["total_votes"].cast(IntegerType()))

In [11]:
print(cameras_df.select("customer_id").dtypes)
print(cameras_df.select("product_parent").dtypes)
print(cameras_df.select("star_rating").dtypes)
print(cameras_df.select("helpful_votes").dtypes)
print(cameras_df.select("total_votes").dtypes)

[('customer_id', 'int')]
[('product_parent', 'int')]
[('star_rating', 'int')]
[('helpful_votes', 'int')]
[('total_votes', 'int')]


In [16]:
cameras_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Count the number of records in the Database

In [17]:
# Row Count
cameras_df.count()

1801849

Transform the dataset to fit the tables in the schema file

In [18]:
from pyspark.sql.functions import to_date
# DataFrame
review_id = cameras_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006EGZ9CPJP5|   42900361|B00M07I2YC|     645536663| 2015-01-01|
| R1014HFB7LT6D|   46413833|B005HY4TXS|     810992030| 2012-12-13|
|R101EWV42EDDZ3|   16227012|B00009R6FV|     408598223| 2014-05-12|
|R102BEG999GARX|   13929688|B001TSSOQY|     330601228| 2012-04-10|
|R103A1CV3F69NS|   47814869|B0048IW030|     990185820| 2013-08-12|
|R104HWMYBVOGGY|   35751178|B000P41DVG|     855766027| 2010-04-12|
|R104JY83ZI0SSX|   12418820|B0044XAXLU|     783331329| 2014-04-15|
|R104UXCOW5V7AF|   21868179|B00005RH7M|      61995094| 2008-06-30|
|R1052PBM1E3ZX6|   21278613|B00OT7W6VQ|     840491020| 2015-03-29|
|R1052RLYZT120F|   23296488|B001196MG0|     250259634| 2012-07-07|
|R10584T58O3B5Y|   44752014|B005W17OPG|     754257369| 2014-02-06|
|R1067QRZHX62NI|   10810208|B0076YO2CA|     468407985| 2014-08

In [19]:
from pyspark import SparkFiles
# DataFrame
products = cameras_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L6CBM4E|Manfrotto MKCOMPA...|
|B00006JPWW|Panasonic AY-DVMC...|
|B0006GDHLC|Energizer(R) 1.5-...|
|B00RELNT2E|Kastar Charger, B...|
|B00554PCDG|Pixel Flash Hot S...|
|B001SER4CO|Canon Powershot A480|
|B0030TYVPI|Traffic Light 4-P...|
|B000MXLAFO|Casio Exilim EXZ7...|
|B0007IOYU2|Olympus 7X21 PC I...|
|B000SMZDIY|FIREWIRE DV CABLE...|
|B003GRHBEM|RainbowImaging Ca...|
|B00F0S56BU|Assurant 2-Year C...|
|B00005LESG|Canon EF 70-200mm...|
|B002VP9M7E|Speck Products IP...|
|B00022OB9W|Mustek MDC-5500Z ...|
|B001JDD1OE|Canon 50mm F/1.8 ...|
|B0009QZDL6|Lastolite LL LR12...|
|B0013FDKRG|Home Base with Fa...|
|B0000BZL0G|B+W Clear UV Haze...|
|B004CBTCFC|Frio ENLFRC1A Hot...|
+----------+--------------------+
only showing top 20 rows



In [20]:
from pyspark import SparkFiles
# DataFrame
customers = cameras_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42553481|            11|
|   14138259|             4|
|   21461921|            11|
|   34969000|             1|
|    9842436|             2|
|   12422616|             3|
|   14154756|             2|
|   43817031|             4|
|   13934198|             1|
|   43627964|             1|
|   45108886|             1|
|   52128920|             2|
|   37439895|             1|
|   31613429|             1|
|   47583650|             1|
|   33078812|             1|
|   12960226|             5|
|   42354937|             3|
|   29423910|             1|
|   28604485|             2|
+-----------+--------------+
only showing top 20 rows



In [21]:
from pyspark import SparkFiles
vine_table = cameras_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006EGZ9CPJP5|          5|            1|          1|   N|
| R1014HFB7LT6D|          5|            5|          5|   N|
|R101EWV42EDDZ3|          5|            1|          1|   N|
|R102BEG999GARX|          4|            0|          5|   N|
|R103A1CV3F69NS|          5|            0|          0|   N|
|R104HWMYBVOGGY|          5|            1|          1|   N|
|R104JY83ZI0SSX|          1|            1|          1|   N|
|R104UXCOW5V7AF|          5|            0|          0|   N|
|R1052PBM1E3ZX6|          5|            0|          0|   N|
|R1052RLYZT120F|          5|            0|          0|   N|
|R10584T58O3B5Y|          4|            0|          0|   N|
|R1067QRZHX62NI|          3|            0|          0|   N|
|R106E4HEWBPHFQ|          5|            2|          2|   N|
|R106XP4LMRZ88N|          5|            

Load the Dataframes that correspond to tables into an RDS instance

In [ ]:
mode = 'append'
jdbc_url = "jdbc:postgresql://mypostgresdb.ceomfsf1omq9.us-east-2.rds.amazonaws.com:5432/my_data_class_db_camera"
config = {
    'user':'postgres',
    'password':'postgres',
    'driver':"org.postgresql.Driver"
}

In [23]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [24]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [25]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [26]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)